In [5]:
import json
import pyodbc
import datetime
from urllib.request import urlopen as uo
from bs4 import BeautifulSoup as soup

In [6]:
def getNewsList():
    #Menentukan URL dari Website yang ingin di Crawl
    base_url = "https://www.surabayatimes.com/feed"

    #Membuat Request ke Website
    uClient = uo(base_url)
    news_page = uClient.read()
    uClient.close()

    #Men-generate file .HTML dari URL
    page_soup = soup(news_page, "xml")
    items = page_soup.findAll("item")
    for i in items:
        href = i.find("link").text
        getNewsData(href)

In [21]:
def getNewsData(news_url):
    conn = pyodbc.connect("DRIVER={SQL Server}; server=mysights.eastus.cloudapp.azure.com; database=OpinionMining; uid=mysights; pwd=zXcasdqwe123")
    c = conn.cursor()
    uClient = uo(news_url)
    news_page = uClient.read()
    uClient.close()
    
    trigger = 0
    trigger_tag = 0
    
    page_soup = soup(news_page, "html.parser")
    news_header = page_soup.find("title").text.split("| ")[0]
    news_category = page_soup.find("title").text.split("- ")[1]
    news_created = datetime.datetime.now()
    news_article = page_soup.find("div", {"id":"content-area"}).text
    try:
        wptrim = page_soup.find("div", {"id":"content-area"}).findAll("div", {"class":"wp-caption-text2"})
        for i in wptrim:
            news_article = news_article.replace(i.text, "").strip()
    except:
        wptrim = ""
    news_author = page_soup.findAll("td")[2].text
    news_editor = page_soup.findAll("td")[5].text
    news_publisher = page_soup.findAll("td")[8].text
    news_source = page_soup.findAll("td")[11].text
    news_timestamp = page_soup.find("span", {"class":"post-byline"}).text.split(", ")[1]
    news_location = ""
    news_id = news_url.split("/")[4]
    tag_url = ""
    news_tags = page_soup.find("div", {"class":"post-tags"}).text.replace("TAG'S", "").replace("#", "").strip().replace("\n", ",")


    print(news_id)
    print(news_created)
    print(news_tags)
    
 

    c.execute("SELECT * FROM news_portal_surabayatimes WHERE news_id=?",news_id)
    for row in c:
        if row.news_id is not None:
            trigger = 1
            
    if trigger == 0:
        c.execute("""INSERT INTO news_portal_surabayatimes (news_id, news_category, news_header, news_author, news_editor, news_timestamp, 
                    news_article, news_url, news_created, news_location) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", 
                  (news_id, news_category, news_header, news_author, news_editor, news_timestamp, 
                    news_article, news_url, news_created, news_location))
        conn.commit()
        print(news_id)
    else:
        print("Data with that ID already existed.")
    trigger = 0
    
    c.execute("SELECT * FROM news_portal_surabayatimes_tag WHERE news_id=?",news_id)
    for row in c:
        if row.news_id is not None:
            trigger_tag = 1
            
    if trigger_tag == 0:
        c.execute("""INSERT INTO news_portal_surabayatimes_tag (news_id, news_tag, news_created, tag_url) VALUES (?, ?, ?, ?)""", 
                  (news_id, news_tags, news_created, tag_url))
        conn.commit()
    else:
        print("Tag with that ID already existed.")
    trigger_tag = 0


In [22]:
if __name__ == "__main__":
    getNewsList()

189125
2019-03-05 15:27:25.761206
berita surabaya, Penerima bantuan iuran BPJS kesehatan yang ditanggung oleh Anggaran Pendapatan Belanja Daerah, Wali Kota Tri Rismaharini, PBI BPJS terbesar di Indonesia
189058
2019-03-05 15:27:30.072678
kediri, berita kediri, Bayi Dibuang Dua Pria, Desa Cendono, Polsek Kandat, bayi yang dibuang
189055
2019-03-05 15:27:35.668713
surabaya, berita surabaya, USBN Pakai Smartphone, Gubernur Khofifah, Ujian Sekolah Berstandar Nasional (USBN)
189029
2019-03-05 15:27:39.186278
surabaya, berita surabaya, Capres, Jokowi Ma'ruf Amin
189022
2019-03-05 15:27:42.561259
surabaya, berita surabaya, Polrestabes Surabaya, Wali Kota Surabaya, Risma
188970
2019-03-05 15:27:48.335825
surabaya, berita surabaya, Bonek Aremania Berdamai, Gubernur Jawa Timur Khofifah Indar Parawansa, Bonek Mania, Aremania
188949
2019-03-05 15:27:54.441506
surabaya, berita surabaya, Stadion Gelora Bung Tomo, Pemkot Surabaya
188938
2019-03-05 15:27:59.508963
surabaya, berita surabaya, Ketua PKK 

In [ ]:
# conn = pyodbc.connect("DRIVER={SQL Server}; server=.; database=news_crawler; uid=; pwd=")
# c = conn.cursor()

# c.execute("insert into portal_news_surabayatimes(news_id, news_category, news_header, news_author, news_editor, news_timestamp, news_article, news_url, news_created)VALUES(?,?,?,?,?,?,?,?,?)",
#           (news_id, news_category, news_header, news_author, news_editor, news_timestamp, news_article, news_url, news_created))
# conn.commit()

NameError: name 'news_id' is not defined